In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from ipywidgets import IntSlider, HBox, interactive_output
import ipywidgets as widgets
import os
import matplotlib.pyplot as plt
from IPython.display import display
from scipy.stats import skew, kurtosis
import numpy as np
from pandas.tseries.holiday import USFederalHolidayCalendar
import seaborn as sns
from functools import reduce

from utils import *

In [2]:
# Load data
data_path = os.path.dirname(os.getcwd()) + '/data'
data_file = data_path + '/fpi_raw_data.xlsx'
data = pd.read_excel(data_file, sheet_name='Universe of broad assets', index_col=0, parse_dates=True)

# Initial Visualizations

In [3]:
def plot_histogram(series, bin_width, start_date, end_date):
    # Filter data by selected date range and series
    series_data = data[series]
    if start_date and end_date:
        filtered_data = series_data[(series_data.index >= pd.to_datetime(start_date)) & 
                                    (series_data.index <= pd.to_datetime(end_date))]
    else:
        filtered_data = series_data
    
    # Plot histogram
    plt.figure(figsize=(10, 6))
    plt.hist(filtered_data.dropna(), bins=bin_width, edgecolor='black')
    plt.title(f'Histogram of {series}')
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.show()

def summary_table(series, start_date, end_date):
    # Filter data by selected date range and series
    series_data = data[series]
    if start_date and end_date:
        filtered_data = series_data[(series_data.index >= pd.to_datetime(start_date)) & 
                                    (series_data.index <= pd.to_datetime(end_date))]
    else:
        filtered_data = series_data
        
    # Display summary statistics
    stats = {
        'Count': filtered_data.count(),
        'Mean': filtered_data.mean(),
        'Standard Deviation': filtered_data.std(),
        'Skewness': skew(filtered_data.dropna()),
        'Kurtosis': kurtosis(filtered_data.dropna()),
        'Minimum': filtered_data.min(),
        'Maximum': filtered_data.max()
    }
    
    # Print stats in a table format
    stats_df = pd.DataFrame(stats, index=[0]).T.rename(columns={0: 'Value'})
    display(stats_df)

In [4]:
# Widget controls
series_selector = widgets.Dropdown(options=data.columns, description='Series')
bin_slider = widgets.IntSlider(value=50, min=10, max=100, description='Bin Count')
start_date_picker = widgets.DatePicker(description='Start Date')
end_date_picker = widgets.DatePicker(description='End Date')

# Display widgets
controls = widgets.HBox([series_selector, bin_slider, start_date_picker, end_date_picker])

# Interactive plot output
out1 = widgets.interactive_output(plot_histogram, {
    'series': series_selector,
    'bin_width': bin_slider,
    'start_date': start_date_picker,
    'end_date': end_date_picker
})

out2 = widgets.interactive_output(summary_table, {
    'series': series_selector,
    'start_date': start_date_picker,
    'end_date': end_date_picker
})

ui = widgets.VBox([controls, out1])
widgets.HBox([ui, out2])


# Aggregating Returns up to Weekly

In [3]:
broad_assets = data.copy()
broad_assets.head()

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2000-07-31,0.015154,0.075677,-0.002266,0.000897,0.042162,0.004952,0.037313,0.002093,-0.002477,-0.008385,-0.031787
2000-08-01,0.009898,-0.049140,0.007874,0.006272,-0.011478,0.040433,-0.029227,0.010693,0.009594,0.000455,0.014613
2000-08-02,0.000718,-0.017754,0.001530,0.003732,0.009817,-0.009512,-0.000527,0.000973,0.011188,0.001176,0.018693
2000-08-03,0.019197,0.076083,0.007209,0.003099,-0.003059,-0.020954,-0.030686,-0.009466,0.006064,-0.026208,0.007214
2000-08-04,0.014248,-0.002954,0.006556,0.006231,0.016388,-0.024891,0.012934,0.004234,-0.001179,-0.003552,0.032614


In [4]:
broad_assets.index = pd.to_datetime(broad_assets.index)
broad_assets['weekday'] = broad_assets.index.day_name()
broad_assets.head() #['2000-08-27':'2000-09-10']

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11,weekday
Date,,,,,,,,,,,,
2000-07-31,0.015154,0.075677,-0.002266,0.000897,0.042162,0.004952,0.037313,0.002093,-0.002477,-0.008385,-0.031787,Monday
2000-08-01,0.009898,-0.049140,0.007874,0.006272,-0.011478,0.040433,-0.029227,0.010693,0.009594,0.000455,0.014613,Tuesday
2000-08-02,0.000718,-0.017754,0.001530,0.003732,0.009817,-0.009512,-0.000527,0.000973,0.011188,0.001176,0.018693,Wednesday
2000-08-03,0.019197,0.076083,0.007209,0.003099,-0.003059,-0.020954,-0.030686,-0.009466,0.006064,-0.026208,0.007214,Thursday
2000-08-04,0.014248,-0.002954,0.006556,0.006231,0.016388,-0.024891,0.012934,0.004234,-0.001179,-0.003552,0.032614,Friday


In [7]:
broad_assets['trading_day'] = np.where((broad_assets.weekday == 'Tuesday'), 1, 0)   # Start of return calc periods are typically Tuesday
broad_assets['trading_day'] = np.where((broad_assets.weekday == 'Wednesday') & 
                                       (broad_assets.weekday.shift(1) == 'Monday'), 
                                       1, broad_assets.trading_day) # This filter sets Wednesday as the first trading day when Tuesday is a holiday
broad_assets['trading_day'] = np.where((broad_assets.weekday == 'Wednesday') & 
                                       (broad_assets.weekday.shift(1) == 'Friday'), 
                                       1, broad_assets.trading_day) # There are a few cases where markets were shut down both Monday and Tuesday
# The 9/11 Terrorist attacks were on a Tuesday and shut down markets for a week. Markets re-opened on 9/17, so assuming we place trades 
# that day and calculate returns through the following EOD Monday. This is the longest holding window in the dataset.
# TODO: Question, should we just drop 9/17/2001 from the dataset?
broad_assets['trading_day'] = np.where(broad_assets.index == '2001-09-17', 1, broad_assets.trading_day)
broad_assets['trading_day'] = np.where(broad_assets.index == '2001-09-18', 0, broad_assets.trading_day)
broad_assets['week'] = broad_assets.trading_day.cumsum()
broad_assets['2000-08-27':'2000-09-12']

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11,weekday,trading_day,week
Date,,,,,,,,,,,,,,
2000-08-28,0.009907,0.011431,-0.011551,-0.006292,0.004840,-0.004696,0.017255,-0.007905,0.003144,-0.000994,0.030595,Monday,0,4
2000-08-29,-0.005726,-0.001428,-0.005991,-0.004429,0.011857,-0.016923,-0.028685,0.004255,0.013718,-0.005372,0.007735,Tuesday,1,5
2000-08-30,-0.009353,0.008240,0.002775,0.000891,0.010276,0.023121,0.003055,-0.012985,-0.000970,0.005220,0.033015,Wednesday,0,5
2000-08-31,0.019846,0.054594,0.014681,0.011561,0.020727,-0.010972,0.013995,-0.001245,0.008997,0.026358,-0.005365,Thursday,0,5
2000-09-01,0.003850,0.010384,0.003376,0.007755,0.014758,0.002049,0.042495,0.000655,-0.021759,-0.008183,-0.003105,Friday,0,5
2000-09-05,-0.018275,-0.055039,0.000885,0.000264,-0.011136,0.010078,-0.017364,-0.004304,0.019478,-0.008936,0.024536,Tuesday,1,6
2000-09-06,-0.019759,-0.075212,-0.009674,-0.005023,-0.009874,0.004712,-0.039244,-0.011284,0.030516,-0.008248,0.017216,Wednesday,0,6
2000-09-07,0.013537,0.060054,-0.004105,-0.003164,0.023932,0.013575,0.017056,-0.018844,-0.007099,-0.005001,0.013816,Thursday,0,6
2000-09-08,-0.010919,-0.071050,0.005894,0.003386,-0.029024,0.031138,-0.042704,-0.007504,0.012403,-0.005743,-0.048884,Friday,0,6


In [8]:
broad_assets.week.value_counts()    # The one long return period is 9/11

week
59      6
1255    5
1254    5
1       5
2       5
       ..
1246    4
1248    4
967     4
976     4
0       1
Name: count, Length: 1258, dtype: int64

In [9]:
# Showing how 9/11 is being handled
broad_assets.loc[(broad_assets.week == 59) | (broad_assets.week == 58)]

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11,weekday,trading_day,week
Date,,,,,,,,,,,,,,
2001-09-04,-0.001256,-0.062164,-0.030723,-0.018517,-0.006977,-0.004223,-0.054898,0.011222,0.039585,-0.020181,-0.015338,Tuesday,1,58
2001-09-05,-0.001850,-0.012526,0.003154,0.003080,-0.019005,-0.005365,-0.010260,-0.014120,-0.001585,0.002805,0.001100,Wednesday,0,58
2001-09-06,-0.044849,-0.075897,0.017594,0.014829,-0.039374,-0.025737,-0.030640,-0.025920,-0.010916,0.007947,0.010400,Thursday,0,58
2001-09-07,-0.037398,-0.011037,0.010878,0.008597,-0.036303,-0.018334,-0.066198,-0.007238,-0.016389,-0.000138,0.013082,Friday,0,58
2001-09-10,0.012343,0.016289,-0.011736,-0.003933,-0.020093,-0.007720,0.038647,-0.017213,0.009577,-0.010384,-0.010573,Monday,0,58
2001-09-17,-0.097882,-0.165190,0.013855,0.039990,-0.104084,-0.075047,-0.115303,-0.032963,-0.042481,0.130143,-0.011503,Monday,1,59
2001-09-18,-0.011715,-0.044917,-0.032494,-0.010723,-0.028863,0.019030,-0.030498,0.010262,-0.003618,-0.011864,-0.039678,Tuesday,0,59
2001-09-19,-0.032236,-0.026910,-0.001095,0.002691,-0.041062,-0.023215,-0.018170,0.014810,-0.004693,0.019003,-0.014230,Wednesday,0,59
2001-09-20,-0.062195,-0.069560,-0.017388,-0.007711,-0.077120,-0.050275,-0.087383,-0.056703,0.005272,-0.012840,-0.009203,Thursday,0,59


# Building indicators

In [5]:
broad_assets = data

In [6]:
weekly_rets = weekly_returns(broad_assets)
weekly_rets.to_excel(data_path + '/broad_assets_weekly_rets.xlsx', sheet_name='broad_assets_weekly')
display(weekly_rets)

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2000-07-31,0.015154,0.075677,-0.002266,0.000897,0.042162,0.004952,0.037313,0.002093,-0.002477,-0.008385,-0.031787
2000-08-07,0.067870,0.052623,0.015929,0.013358,0.036514,-0.023689,-0.019612,-0.011556,0.024090,-0.026565,0.034584
2000-08-14,0.015698,0.002262,0.015185,0.006338,0.017116,0.025353,0.029392,0.036872,0.008525,0.006253,0.048137
2000-08-21,0.009488,0.056625,-0.001838,-0.000252,0.006775,-0.026410,0.001619,-0.002437,0.007701,0.001821,0.038370
2000-08-28,0.018391,0.065276,0.001894,0.004126,0.037931,-0.035294,0.019534,-0.017000,0.001856,-0.009469,0.031318
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-12,0.060689,0.071826,-0.044042,-0.014738,0.020995,0.042517,0.043057,0.088217,0.009326,0.050343,0.058296
2024-08-19,0.100581,0.134610,0.019896,-0.000441,0.103698,0.032142,0.093122,0.077077,-0.023833,0.023948,-0.037742
2024-08-26,0.002016,-0.026628,0.002796,0.006442,0.045003,0.054048,0.029013,-0.017973,-0.019513,0.009670,0.033216


## ROC & MACD Output

In [7]:
asset_columns = [col for col in weekly_rets.columns if col.startswith('Asset')]

In [8]:
roc_df = calculate_roc(weekly_rets, asset_columns, period=52)
roc_df.to_excel(data_path + '/broad_assets_roc_52w.xlsx', sheet_name='broad_assets_roc')
display(roc_df)

,ROC_Asset 1,ROC_Asset 2,ROC_Asset 3,ROC_Asset 4,ROC_Asset 5,ROC_Asset 6,ROC_Asset 7,ROC_Asset 8,ROC_Asset 9,ROC_Asset 10,ROC_Asset 11
Date,,,,,,,,,,,
2001-07-23,-0.347629,-0.853581,0.169732,0.230976,-0.109603,0.069124,-0.412476,-0.529694,0.216806,-0.130331,-0.036000
2001-07-30,-0.343450,-0.857381,0.180953,0.238395,-0.139216,0.036921,-0.417244,-0.520707,0.207861,-0.147093,0.005593
2001-08-06,-0.389394,-0.859408,0.146458,0.213536,-0.182807,0.101045,-0.392876,-0.502361,0.163423,-0.107917,-0.004001
2001-08-13,-0.408055,-0.868186,0.155501,0.227153,-0.207985,0.101466,-0.420267,-0.526455,0.126492,-0.059447,-0.028688
2001-08-20,-0.433315,-0.892783,0.172966,0.235127,-0.209522,0.179081,-0.431420,-0.542531,0.086476,-0.061468,-0.108018
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-12,0.359338,0.387579,0.007113,0.042823,0.076363,0.198900,0.150075,0.120983,0.032570,0.559093,-0.103947
2024-08-19,0.558213,0.633121,0.089741,0.060128,0.273052,0.333888,0.315317,0.264998,0.004960,0.618892,-0.122203
2024-08-26,0.539557,0.568210,0.042993,0.055828,0.313442,0.338275,0.332795,0.193945,-0.029462,0.593793,-0.112936


In [9]:
macd_df = calculate_macd(broad_assets, asset_columns)
macd_df.to_excel(data_path + '/broad_assets_macd.xlsx', sheet_name='broad_assets_macd')
display(macd_df)

,MACD_Line_Asset 1,Signal_Line_Asset 1,MACD_Histogram_Asset 1,MACD_Line_Asset 2,Signal_Line_Asset 2,MACD_Histogram_Asset 2,MACD_Line_Asset 3,Signal_Line_Asset 3,MACD_Histogram_Asset 3,MACD_Line_Asset 4,...,MACD_Histogram_Asset 8,MACD_Line_Asset 9,Signal_Line_Asset 9,MACD_Histogram_Asset 9,MACD_Line_Asset 10,Signal_Line_Asset 10,MACD_Histogram_Asset 10,MACD_Line_Asset 11,Signal_Line_Asset 11,MACD_Histogram_Asset 11
Date,,,,,,,,,,,,,,,,,,,,,
2000-07-31,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2000-08-01,-0.000419,-0.000084,-0.000335,-0.009957,-0.001991,-0.007965,0.000809,0.000162,0.000647,0.000429,...,0.000549,0.000963,0.000193,0.000770,0.000705,0.000141,0.000564,0.003701,0.000740,0.002961
2000-08-02,-0.001475,-0.000362,-0.001113,-0.015141,-0.004621,-0.010519,0.000927,0.000315,0.000612,0.000557,...,0.000243,0.001833,0.000521,0.001313,0.001307,0.000374,0.000933,0.006885,0.001969,0.004916
2000-08-03,-0.000812,-0.000452,-0.000360,-0.011544,-0.006006,-0.005538,0.001462,0.000544,0.000918,0.000601,...,-0.000630,0.002086,0.000834,0.001252,-0.000421,0.000215,-0.000636,0.008384,0.003252,0.005132
2000-08-04,-0.000677,-0.000497,-0.000180,-0.014899,-0.007785,-0.007115,0.001813,0.000798,0.001015,0.000878,...,-0.000270,0.001682,0.001003,0.000679,0.000038,0.000180,-0.000142,0.011490,0.004900,0.006590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-30,0.001068,0.001229,-0.000161,0.000720,0.001044,-0.000325,-0.003007,-0.001391,-0.001616,-0.001097,...,-0.000979,0.001475,0.000077,0.001398,-0.001491,-0.000035,-0.001456,-0.000322,0.001060,-0.001382
2024-09-03,-0.002749,0.000433,-0.003182,-0.004594,-0.000083,-0.004510,-0.000733,-0.001260,0.000527,-0.000322,...,-0.003371,0.001535,0.000368,0.001167,-0.002108,-0.000450,-0.001659,-0.003284,0.000191,-0.003475
2024-09-04,-0.002598,-0.000173,-0.002425,-0.003995,-0.000866,-0.003129,0.001255,-0.000757,0.002012,0.000438,...,-0.002191,0.000612,0.000417,0.000195,-0.001720,-0.000704,-0.001016,-0.003921,-0.000631,-0.003290


## Relative Strength Index

In [10]:
rsi_df = calculate_rsi(broad_assets)
rsi_df.to_excel(data_path + '/broad_assets_rsi.xlsx', sheet_name='broad_assets_rsi')
display(rsi_df)

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2000-08-17,76.286303,70.839717,65.740894,65.285290,72.230173,52.366857,65.717757,73.405647,58.106075,45.660577,63.145762
2000-08-18,71.922049,63.282788,70.150819,67.696703,64.803914,45.805349,54.971430,62.338973,67.261046,46.087018,73.622074
2000-08-21,71.960180,72.744395,63.183382,62.288132,69.426565,27.285297,62.138675,54.992505,66.764292,41.429862,73.258685
2000-08-22,71.067812,76.398216,63.458694,60.669616,68.340172,29.203787,60.889715,56.930387,64.496720,36.927898,66.983885
2000-08-23,69.647360,73.338435,64.512990,64.147901,69.816256,29.469405,74.497313,56.268605,57.861120,39.496834,68.377388
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-30,78.729066,69.349281,47.741951,47.701081,73.854675,84.388412,87.623309,62.256224,38.603754,56.283343,41.282430
2024-09-03,58.656220,49.294752,51.620424,48.394039,57.888671,82.631055,68.004611,48.055030,44.375370,55.627548,37.916214
2024-09-04,56.094431,48.367294,53.314235,53.179980,59.032375,82.283588,64.076768,50.276651,40.779779,60.864825,37.380809


## One-Period Lagged Weekly Returns

In [11]:
lag1_weekly_rets = weekly_rets.shift(1).dropna()
lag1_weekly_rets.to_excel(data_path + '/broad_assets_lag1_rets.xlsx', sheet_name='broad_assets_lag1_rets')
display(lag1_weekly_rets)

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2000-08-07,0.015154,0.075677,-0.002266,0.000897,0.042162,0.004952,0.037313,0.002093,-0.002477,-0.008385,-0.031787
2000-08-14,0.067870,0.052623,0.015929,0.013358,0.036514,-0.023689,-0.019612,-0.011556,0.024090,-0.026565,0.034584
2000-08-21,0.015698,0.002262,0.015185,0.006338,0.017116,0.025353,0.029392,0.036872,0.008525,0.006253,0.048137
2000-08-28,0.009488,0.056625,-0.001838,-0.000252,0.006775,-0.026410,0.001619,-0.002437,0.007701,0.001821,0.038370
2000-09-01,0.018391,0.065276,0.001894,0.004126,0.037931,-0.035294,0.019534,-0.017000,0.001856,-0.009469,0.031318
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-12,-0.101168,-0.122107,0.126673,0.049383,-0.171306,-0.027594,-0.073715,-0.086162,-0.035065,0.020789,-0.024322
2024-08-19,0.060689,0.071826,-0.044042,-0.014738,0.020995,0.042517,0.043057,0.088217,0.009326,0.050343,0.058296
2024-08-26,0.100581,0.134610,0.019896,-0.000441,0.103698,0.032142,0.093122,0.077077,-0.023833,0.023948,-0.037742


## EWMA Volatility

In [12]:
ewma_vol = calc_ewma_volatility(broad_assets, return_columns=asset_columns) * np.sqrt(7)    # scaling daily EWMA vol up to weekly
ewma_vol.to_excel(data_path + '/broad_assets_ewma_vol.xlsx', sheet_name='broad_assets_ewma_vol')
display(ewma_vol)

,EWMA_vol_Asset 1,EWMA_vol_Asset 2,EWMA_vol_Asset 3,EWMA_vol_Asset 4,EWMA_vol_Asset 5,EWMA_vol_Asset 6,EWMA_vol_Asset 7,EWMA_vol_Asset 8,EWMA_vol_Asset 9,EWMA_vol_Asset 10,EWMA_vol_Asset 11
Date,,,,,,,,,,,
2000-07-31,0.033777,0.058735,0.032351,0.032323,0.042321,0.032477,0.040363,0.032346,0.032358,0.032772,0.038325
2000-08-01,0.033370,0.065245,0.031778,0.031601,0.041700,0.040964,0.043477,0.032117,0.031982,0.031775,0.038345
2000-08-02,0.032357,0.064296,0.030826,0.030734,0.040927,0.040192,0.042153,0.031145,0.031844,0.030816,0.039101
2000-08-03,0.033748,0.079480,0.030250,0.029865,0.039730,0.041266,0.045451,0.030814,0.031123,0.034367,0.038197
2000-08-04,0.033998,0.077083,0.029634,0.029235,0.039957,0.043138,0.044857,0.030001,0.030185,0.033400,0.042641
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-30,0.052709,0.075070,0.046386,0.018779,0.080107,0.048647,0.044071,0.053673,0.021510,0.049062,0.049945
2024-09-03,0.057995,0.083477,0.047277,0.018897,0.087384,0.047181,0.049434,0.058099,0.020933,0.047891,0.054680
2024-09-04,0.056270,0.080976,0.048613,0.019365,0.084760,0.045828,0.048043,0.056331,0.021144,0.046451,0.054361


## Skew & Kurtosis

In [13]:
# broad_assets = broad_assets.drop(columns=['weekday', 'trading_day', 'week'])

In [14]:
skew_daily = broad_assets.expanding(min_periods=(252*3)).skew().dropna()
skew_daily.to_excel(data_path + '/broad_assets_skew.xlsx', sheet_name='broad_assets_skew')
display(skew_daily)

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2003-08-05,0.313385,0.623741,-0.629047,-0.499734,0.124680,0.045532,0.168587,-0.147747,0.080812,0.590040,-0.315373
2003-08-06,0.312957,0.625372,-0.586231,-0.474074,0.126565,0.045239,0.168471,-0.146249,0.080355,0.590917,-0.313850
2003-08-07,0.311045,0.625421,-0.588038,-0.475920,0.126729,0.043443,0.167612,-0.148525,0.082872,0.590001,-0.315764
2003-08-08,0.310134,0.626690,-0.587110,-0.476670,0.126712,0.044223,0.165859,-0.150693,0.080286,0.586257,-0.315613
2003-08-11,0.309292,0.625090,-0.583470,-0.473254,0.124164,0.044018,0.164921,-0.152303,0.082700,0.582624,-0.314946
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-30,-0.158617,0.299492,-0.053239,0.049695,-0.298700,0.193631,-0.254603,0.514300,0.012554,-0.178849,-0.327083
2024-09-03,-0.158540,0.299034,-0.053480,0.049357,-0.298815,0.193630,-0.254249,0.514461,0.012422,-0.178652,-0.326822
2024-09-04,-0.158472,0.299128,-0.053636,0.049045,-0.298763,0.193595,-0.254172,0.514523,0.012747,-0.178697,-0.326487


In [15]:
kurtosis_daily = broad_assets.expanding(min_periods=(252*3)).kurt().dropna()
kurtosis_daily.to_excel(data_path + '/broad_assets_kurtosis.xlsx', sheet_name='broad_assets_kurtosis')
display(kurtosis_daily)

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2003-08-05,1.064769,2.664918,0.839847,1.479471,0.249372,2.314086,1.839852,0.908373,0.586300,5.096740,1.089295
2003-08-06,1.069615,2.671631,0.890333,1.482051,0.251814,2.320987,1.846150,0.911492,0.590832,5.107460,1.091987
2003-08-07,1.071052,2.678691,0.894862,1.486541,0.256123,2.324512,1.851561,0.912458,0.589429,5.115103,1.087192
2003-08-08,1.075147,2.686104,0.898024,1.492586,0.260399,2.331044,1.854526,0.914098,0.582135,5.094669,1.092344
2003-08-11,1.079368,2.688712,0.888934,1.483409,0.259313,2.337990,1.859787,0.917809,0.577395,5.057493,1.096865
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-30,10.542040,8.363352,3.528118,3.500898,6.177960,21.328072,9.295014,18.891729,1.613734,5.815555,3.193553
2024-09-03,10.531799,8.354074,3.525848,3.499960,6.169297,21.332069,9.289894,18.885478,1.614377,5.816402,3.189825
2024-09-04,10.533893,8.355895,3.522943,3.498054,6.170708,21.335983,9.291733,18.889088,1.613968,5.817852,3.189445
